# A Movie Recommendation Service
### Source: https://www.codementor.io/@jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw

#### Create a SparkContext configured for local mode


In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

#### File download
Small: 100,000 ratings and 2,488 tag applications applied to 8,570 movies by 706 users. Last updated 4/2015.   
Full: 21,000,000 ratings and 470,000 tag applications applied to 27,000 movies by 230,000 users. Last updated 4/2015.

In [3]:
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'


#### Download location(s)


In [4]:
import os
datasets_path = os.path.join('/home/jovyan', 'work')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')


#### Getting file(s)


In [5]:
import urllib.request
small_f = urllib.request.urlretrieve (small_dataset_url, small_dataset_path)

#### Extracting file(s)


In [6]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

## Loading and parsing datasets
Now we are ready to read in each of the files and create an RDD consisting of parsed lines. 

Each line in the ratings dataset (ratings.csv) is formatted as: 
+ userId,movieId,rating,timestamp 

Each line in the movies (movies.csv) dataset is formatted as:
+ movieId,title,genres 

The format of these files is uniform and simple, so we can use Python split() to parse their lines once they are loaded into RDDs. Parsing the movies and ratings files yields two RDDs: 
+ For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). We drop the timestamp because we do not need it for this recommender.
+ For each line in the movies dataset, we create a tuple of (MovieID, Title). We drop the genres because we do not use them for this recommender.

#### ratings.csv


In [7]:
complete_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]
# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

print ('There are {} recommendations in the complete dataset'.format(complete_ratings_data.count()))
complete_ratings_data.take(3)


There are 100836 recommendations in the complete dataset


[(1, 1, 4.0), (1, 3, 4.0), (1, 6, 4.0)]

#### movies.csv


In [8]:
# Load the small dataset file
complete_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
print ('There are {} movies in the complete dataset'.format(complete_movies_titles.count()))
complete_movies_data.take(3)

There are 9742 movies in the complete dataset


[(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance')]

## Collaborative Filtering
In Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly. 

At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has the following parameters:

+ numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
+ rank is the number of latent factors in the model.
+ iterations is the number of iterations to run.
+ lambda specifies the regularization parameter in ALS.
+ implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
+ alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

#### Selecting ALS parameters using the small dataset
In order to determine the best ALS parameters, we will use the small dataset. We need first to split it into train, validation, and test datasets.

In [9]:
# source uses see=0L, which is the previous version of python (2.x)
# 0L should be written as 0 from now on
training_RDD, validation_RDD, test_RDD = complete_ratings_data.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

#### Training phase


In [10]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ('For rank {} the RMSE is {}'.format(rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank {}'.format(best_rank))

For rank 4 the RMSE is 0.908078105265682
For rank 8 the RMSE is 0.916462973348527
For rank 12 the RMSE is 0.917665030756129
The best model was trained with rank 4


## Using the complete dataset to build the final model
Due to the limitations of virtual machine, we keep using the small dataset instead of complete dataset

We need first to split it into training and test datasets.

In [11]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, \
                           iterations=iterations, lambda_=regularization_parameter)

Now we test on our testing set.


In [12]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

print ('For testing data the RMSE is {}'.format(error))

For testing data the RMSE is 0.8949959237223808


## How to make recommendations
Although we aim at building an online movie recommender, now that we know how to have our recommender model ready, we can give it a try providing some movie recommendations. This will help us coding the recommending engine later on when building the web service, and will explain how to use the model in any other circumstances.

When using collaborative filtering, getting recommendations is not as simple as predicting for the new entries using a previously generated model. Instead, we need to train again the model but including the new user preferences in order to compare them with other users in the dataset. That is, the recommender needs to be trained every time we have new user ratings (although a single model can be used by multiple users of course!). This makes the process expensive, and it is one of the reasons why scalability is a problem (and Spark a solution!). Once we have our model trained, we can reuse it to obtain top recomendations for a given user or an individual rating for a particular movie. These are less costly operations than training the model itself.

Another thing we want to do, is give recommendations of movies with a certain minimum number of ratings. For that, we need to count the number of ratings per movie.

In [13]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

### Adding new user ratings
Now we need to rate some movies for the new user. We will put them in a new RDD and we will use the user ID 0, that is not assigned in the MovieLens dataset. Check the dataset movies file for ID to Tittle assignment (so you know what movies are you actually rating).


In [56]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)

# ###################################################
# Keep the userID, but Replace movieID, rating, title
# ###################################################

# Find 10 movies you have watched in the past
# Put your OWN ratings

# new_user_ratings = [
#      (0,260,4), # Star Wars (1977)
#      (0,1,3), # Toy Story (1995)
#      (0,16,3), # Casino (1995)
#      (0,25,4), # Leaving Las Vegas (1995)
#      (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
#      (0,335,1), # Flintstones, The (1994)
#      (0,379,1), # Timecop (1994)
#      (0,296,3), # Pulp Fiction (1994)
#      (0,858,5), # Godfather, The (1972)
#      (0,50,4) # Usual Suspects, The (1995)
#     ]

# Adding custom Ratings
# new_user_ratings = [
#      (0,100,3), # City Hall (1996)
#      (0,311,1), # Relative Fear (1994)
#      (0,313,2), # Swan Princess, The (1994)
#      (0,1054,4), # Get on the Bus (1996)
#      (0,1027,4), # Robin Hood: Prince of Thieves (1991)
#      (0,1032,4), # Alice in Wonderland (1951)
#      (0,1036,3), # Die Hard (1988)
#      (0,1040,3), # Secret Agent, The (1996)
#      (0,1046,5), # Beautiful Thing (1996)
#      (0,1101,2) # Top Gun (1986)
#     ]

#S1
# new_user_ratings = [
#      (0,100,3), # City Hall (1996)
#      (0,311,1), # Relative Fear (1994)
#      (0,313,2), # Swan Princess, The (1994)
#      (0,5500,2), # Top Secret! (1984)
#      (0,5563,1), # City by the Sea (2002)
#      (0,5628,3), # Wasabi (2001)
#      (0,1036,3), # Die Hard (1988)
#      (0,1040,3), # Secret Agent, The (1996)
#      (0,5755,1), # Happy Birthday to Me (1981)
#      (0,1101,2) # Top Gun (1986)
#     ]

#S2
# new_user_ratings = [
#      (0,1064,4), # Aladdin and the King of Thieves (1996)
#      (0,1032,4), # Alice in Wonderland (1951)
#      (0,1036,3), # Die Hard (1988)
#      (0,1054,4), # Get on the Bus (1996)
#      (0,1027,4), # Robin Hood: Prince of Thieves (1991)
#      (0,1073,5), # Willy Wonka & the Chocolate Factory (1971)
#      (0,999,3), # 2 Days in the Valley (1996)
#      (0,1040,3), # Secret Agent, The (1996)
#      (0,1046,5), # Beautiful Thing (1996)
#      (0,4896,5) # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
#     ]

#S3
new_user_ratings = [
     (0,260,4), # Star Wars (1977)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5), # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]

new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: {}'.format(new_user_ratings_RDD.take(10)))

New user ratings: [(0, 260, 4), (0, 1, 3), (0, 16, 3), (0, 25, 4), (0, 32, 4), (0, 335, 1), (0, 379, 1), (0, 296, 3), (0, 858, 5), (0, 50, 4)]


Now we add them to the data we will use to train our recommender model. We use Spark's union() transformation for this.

In [58]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)


And finally we train the ALS model using all the parameters we selected before (when using the small dataset).

In [59]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed,
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ('New model trained in {} seconds'.format(round(tt,3)))

New model trained in 2.11 seconds


## Getting top recommendations
Let's now get some recommendations! For that we will get an RDD with all the movies the new user hasn't rated yet. We will them together with the model to predict ratings.

In [60]:
# new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # original version: get just movie IDs
new_user_ratings_ids = list(map(lambda x: x[1], new_user_ratings)) # fixed version: get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

We have our recommendations ready. Now we can print out the 25 movies with the highest predicted ratings. And join them with the movies RDD to get the titles, and ratings count in order to get movies with a minimum number of counts. First we will do the join and see what does the result looks like.

In [61]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(81132, ((2.955288581807401, 'Rubber (2010)'), 2)),
 (60408,
  ((2.636138164903529,
    "Welcome to the Sticks (Bienvenue chez les Ch'tis) (2008)"),
   2)),
 (204, ((0.7783204206395666, 'Under Siege 2: Dark Territory (1995)'), 30))]

So we need to flat this down a bit in order to have (Title, Rating, Ratings Count).


In [62]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

Finally, get the highest rated recommendations for the new user, filtering out movies with less than 25 ratings.

In [63]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n{}'.format('\n'.join(map(str, top_movies))))

TOP recommended movies (with more than 25 reviews):
('"Producers', 4.413715167197228, 33)
('Citizen Kane (1941)', 4.330399918478984, 69)
('12 Angry Men (1957)', 4.286009440418237, 57)
('Birdman: Or (The Unexpected Virtue of Ignorance) (2014)', 4.246380665024124, 26)
('Strangers on a Train (1951)', 4.227688009723412, 25)
('"Philadelphia Story', 4.22428406396692, 29)
('"Boot', 4.223205521286304, 40)
('Apocalypse Now (1979)', 4.208841648856371, 107)
('"Bridge on the River Kwai', 4.191475180447089, 45)
('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 4.177194898317937, 97)
('Chinatown (1974)', 4.148454095199036, 59)
('Raging Bull (1980)', 4.102537660743968, 40)
('"Maltese Falcon', 4.087109344558157, 44)
('Seven Samurai (Shichinin no samurai) (1954)', 4.084639969466412, 48)
('"Godfather: Part II', 4.073786810835896, 129)
('"African Queen', 4.062454498618735, 34)
('Lawrence of Arabia (1962)', 4.055775606343244, 45)
('Rear Window (1954)', 4.023139250841821, 84)


## Getting individual ratings
Another useful usecase is getting the predicted rating for a particular movie for a given user. The process is similar to the previous retreival of top recommendations but, instead of using predcitAll with every single movie the user hasn't rated yet, we will just pass the method a single entry with the movie we want to predict the rating for.

In [64]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
# individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD = new_ratings_model.predictAll(my_movie)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=500, rating=2.231213731088749)]

Sample Output of Top 10 Recommendations, filtering out movies with less than 50 ratings (meaning with 50 reviews or more):

In [65]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=50).takeOrdered(10, key=lambda x: -x[1])

print ('TOP 10 recommended movies (with more than 50 reviews):\n{}'.format('\n'.join(map(str, top_movies))))

TOP 10 recommended movies (with more than 50 reviews):
('Citizen Kane (1941)', 4.330399918478984, 69)
('12 Angry Men (1957)', 4.286009440418237, 57)
('Apocalypse Now (1979)', 4.208841648856371, 107)
('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 4.177194898317937, 97)
('Chinatown (1974)', 4.148454095199036, 59)
('"Godfather: Part II', 4.073786810835896, 129)
('Rear Window (1954)', 4.023139250841821, 84)
('This Is Spinal Tap (1984)', 4.013146562602245, 66)
('Brazil (1985)', 4.00697399862042, 59)
('2001: A Space Odyssey (1968)', 3.9727756276992223, 109)


Sample Output of Top 25 Recommendations, filtering out movies with less than 25 ratings (meaning with 25 reviews or more):

In [66]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP 25 recommended movies (with more than 25 reviews):\n{}'.format('\n'.join(map(str, top_movies))))

TOP 25 recommended movies (with more than 25 reviews):
('"Producers', 4.413715167197228, 33)
('Citizen Kane (1941)', 4.330399918478984, 69)
('12 Angry Men (1957)', 4.286009440418237, 57)
('Birdman: Or (The Unexpected Virtue of Ignorance) (2014)', 4.246380665024124, 26)
('Strangers on a Train (1951)', 4.227688009723412, 25)
('"Philadelphia Story', 4.22428406396692, 29)
('"Boot', 4.223205521286304, 40)
('Apocalypse Now (1979)', 4.208841648856371, 107)
('"Bridge on the River Kwai', 4.191475180447089, 45)
('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 4.177194898317937, 97)
('Chinatown (1974)', 4.148454095199036, 59)
('Raging Bull (1980)', 4.102537660743968, 40)
('"Maltese Falcon', 4.087109344558157, 44)
('Seven Samurai (Shichinin no samurai) (1954)', 4.084639969466412, 48)
('"Godfather: Part II', 4.073786810835896, 129)
('"African Queen', 4.062454498618735, 34)
('Lawrence of Arabia (1962)', 4.055775606343244, 45)
('Rear Window (1954)', 4.023139250841821, 8

### Which scenario works best with collaborative filtering? Why does this happen?

The scenario that likely worked best for me with collaborative filtering is S3: 10 movies you have watched with ratings balanced/mixed with lower and higher ratings.

The reason for this is that collaborative filtering algorithms, like the ALS algorithm, rely on finding patterns and similarities in user preferences based on their ratings. When a user has a diverse range of ratings, both high and low, it provides more information to the algorithm to capture the user's preferences accurately.

For S1, I have only provided low ratings (3 or lower) for the movies watched. This limited range of ratings may not provide enough information for the collaborative filtering algorithm to accurately capture the user's preferences and find similar users or items.The algorithm may struggle to differentiate between the user's dislikes and their actual preferences for movies they might potentially enjoy.

For S2,I have only provided low ratings (3 or higher) for the movies watched. While this provides more positive feedback than scenario S1, it still lacks diversity in the ratings. The algorithm may have difficulty distinguishing between the user's preferences for different types of movies or identifying areas where the user might not enjoy certain genres or styles.

For S3, the I have provided a diverse range of ratings, including both high and low scores. This diversity in ratings provides more information for the collaborative filtering algorithm to learn the user's preferences accurately. The algorithm can identify patterns and similarities between the user's ratings and those of other users, as well as between the user's ratings and the characteristics of the movies themselves. Having a mix of high and low ratings allows the algorithm to better differentiate between the user's likes and dislikes, leading to more accurate recommendations.

### How would you design your movie streaming service (user interfaces with interactions) to get customer’s input which improves your recommendation engine better in movie searching and selection? (as if you are running a service like Netflix).

To design a movie streaming service that improves the recommendation engine for better movie searching and selection, the user interfaces and interactions should focus on gathering comprehensive user data and preferences. First, personalized user profiles with explicit rating systems and activity tracking mechanisms can capture users' movie preferences, viewing patterns, and engagement levels.

Second, interactive movie discovery features, such as advanced search filters, recommendation carousels, watchlists, and social sharing capabilities, enable users to explore new content while providing implicit feedback through their interactions.

Third, contextual recommendations that consider factors like location, time, device, and mood can offer more relevant suggestions. Additionally, dedicated personalized recommendation pages with feedback mechanisms allow users to refine and improve the recommendations over time.

Finally, implementing intelligent search and recommendation algorithms that leverage natural language processing and machine learning can enhance the accuracy and relevance of recommendations. Continuously gathering user feedback and suggestions can further inform the iterative improvement of the recommendation engine and user experience.

### Copy and paste all three pairs of each scenario and top recommended movies (with more than 25 reviews)


#### S1: 10 movies you have watched with ratings 3 or lower

TOP 25 recommended movies (with more than 25 reviews):
('"Grand Day Out with Wallace and Gromit', 3.418072993307164, 28)
('Old Boy (2003)', 3.2921878198398584, 39)
('Raging Bull (1980)', 3.2859447121868683, 40)
('Brazil (1985)', 3.281084391840988, 59)
('"Thing', 3.248026369170306, 45)
('There Will Be Blood (2007)', 3.2447450216307896, 28)
('Evil Dead II (Dead by Dawn) (1987)', 3.2429132231950883, 34)
('Fear and Loathing in Las Vegas (1998)', 3.2408693995231603, 46)
("One Flew Over the Cuckoo's Nest (1975)", 3.2359559058496856, 133)
('Kill Bill: Vol. 2 (2004)', 3.2060013556560523, 110)
('"Apartment', 3.2023094734513116, 27)
('Raising Arizona (1987)', 3.195010414781218, 58)
('Trainspotting (1996)', 3.1879430261943487, 102)
('Platoon (1986)', 3.1766455539081155, 63)
('Birdman: Or (The Unexpected Virtue of Ignorance) (2014)', 3.165217401217758, 26)
('Dazed and Confused (1993)', 3.1570036350872925, 42)
('Seven Samurai (Shichinin no samurai) (1954)', 3.1561643291275887, 48)
('Young Frankenstein (1974)', 3.1472364796551267, 69)
('Taxi Driver (1976)', 3.1413093247205346, 104)
('Spirited Away (Sen to Chihiro no kamikakushi) (2001)', 3.1396030458221, 87)
('Blade Runner (1982)', 3.138918284796169, 124)
('"Godfather', 3.131670616722417, 192)
('Blue Velvet (1986)', 3.1310501698642437, 46)
('Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)', 3.1265624126287452, 27)
('"Big Lebowski', 3.1153639207568062, 106)

#### S2: 10 movies you have watched with ratings 3 or higher

TOP 25 recommended movies (with more than 25 reviews):
('Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 5.181006998623129, 34)
('Argo (2012)', 5.084689625191196, 28)
("Singin' in the Rain (1952)", 5.04965286411575, 47)
('"Great Escape', 5.025653702944943, 43)
('Casablanca (1942)', 4.985975083447752, 100)
('To Kill a Mockingbird (1962)', 4.9385636193514575, 58)
("Breakfast at Tiffany's (1961)", 4.931641375575502, 36)
('Patton (1970)', 4.89159529201835, 33)
('Wallace & Gromit: A Close Shave (1995)', 4.8765148944658065, 48)
('"Sound of Music', 4.876353538885464, 64)
('Wallace & Gromit: The Wrong Trousers (1993)', 4.863375226377155, 56)
('My Fair Lady (1964)', 4.842977779685219, 35)
('Harry Potter and the Deathly Hallows: Part 1 (2010)', 4.824187948002976, 47)
('Harry Potter and the Deathly Hallows: Part 2 (2011)', 4.8236964945173995, 50)
('"Wizard of Oz', 4.822332781366246, 92)
('"Shawshank Redemption', 4.819798864010686, 317)
('21 Jump Street (2012)', 4.808630202737689, 26)
('Forrest Gump (1994)', 4.806849008373366, 329)
('Mary Poppins (1964)', 4.806264696339085, 71)
('X-Men: First Class (2011)', 4.8048897488170645, 43)
("Schindler's List (1993)", 4.768656621392201, 220)
('Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)', 4.767930528383605, 27)
('The Imitation Game (2014)', 4.7649882539238, 50)
('Wallace & Gromit: The Best of Aardman Animation (1996)', 4.760963492510496, 27)
('Star Wars: Episode VII - The Force Awakens (2015)', 4.757058678472715, 41)

#### S3: 10 movies you have watched with ratings balanced/mixed with lower and higher ratings

TOP 25 recommended movies (with more than 25 reviews):
('"Producers', 4.413715167197228, 33)
('Citizen Kane (1941)', 4.330399918478984, 69)
('12 Angry Men (1957)', 4.286009440418237, 57)
('Birdman: Or (The Unexpected Virtue of Ignorance) (2014)', 4.246380665024124, 26)
('Strangers on a Train (1951)', 4.227688009723412, 25)
('"Philadelphia Story', 4.22428406396692, 29)
('"Boot', 4.223205521286304, 40)
('Apocalypse Now (1979)', 4.208841648856371, 107)
('"Bridge on the River Kwai', 4.191475180447089, 45)
('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 4.177194898317937, 97)
('Chinatown (1974)', 4.148454095199036, 59)
('Raging Bull (1980)', 4.102537660743968, 40)
('"Maltese Falcon', 4.087109344558157, 44)
('Seven Samurai (Shichinin no samurai) (1954)', 4.084639969466412, 48)
('"Godfather: Part II', 4.073786810835896, 129)
('"African Queen', 4.062454498618735, 34)
('Lawrence of Arabia (1962)', 4.055775606343244, 45)
('Rear Window (1954)', 4.023139250841821, 84)
('Blue Velvet (1986)', 4.021742695323522, 46)
('This Is Spinal Tap (1984)', 4.013146562602245, 66)
('Brazil (1985)', 4.00697399862042, 59)
('Harold and Maude (1971)', 3.9999792514768453, 26)
('Sling Blade (1996)', 3.978021042181884, 47)
('2001: A Space Odyssey (1968)', 3.9727756276992223, 109)
('Old Boy (2003)', 3.9636490180691153, 39)